In [ ]:
import tkinter as tk
from tkinter import *
from cv2 import *
import PIL
from PIL import Image,ImageTk
from tkinter.filedialog import askopenfilename
window = tk.Tk()
window.title("License Plate Recognition")
canvas = tk.Canvas(window)
canvas.pack(side=RIGHT)
btn_blur=tk.Button(window, text="Load Image",width=50,height=2,command=selectfile)
btn_blur.pack(anchor=tk.CENTER, expand=True)
result=tk.Label(window,text="Result Here")
result.config(font=("Courier", 44))
objects=tk.Label(window,text="")
objects.config(font=("Courier",30))
extract=tk.Label(window,text="")
objects.pack()
extract.pack()
result.pack()
window.mainloop()

In [ ]:
import numpy as np

class ObjectDetection(object):
    """Class for Custom Vision's exported object detection model
    """

    ANCHORS = np.array([[0.573, 0.677], [1.87, 2.06], [3.34, 5.47], [7.88, 3.53], [9.77, 9.17]])
    IOU_THRESHOLD = 0.45

    def __init__(self, labels, prob_threshold = 0.10, max_detections = 20):
        """Initialize the class

        Args:
            labels ([str]): list of labels for the exported model.
            prob_threshold (float): threshold for class probability.
            max_detections (int): the max number of output results.
        """ 

        assert len(labels) >= 1, "At least 1 label is required"

        self.labels = labels
        self.prob_threshold = prob_threshold
        self.max_detections = max_detections

    def _logistic(self, x):
        return np.where(x > 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

    def _non_maximum_suppression(self, boxes, class_probs, max_detections):
        """Remove overlapping bouding boxes
        """
        assert len(boxes) == len(class_probs)
        if len(boxes)<max_detections:
            max_detections=len(boxes)
        max_probs = np.amax(class_probs, axis=1)
        max_classes = np.argmax(class_probs, axis=1)

        areas = boxes[:,2] * boxes[:,3]

        selected_boxes = []
        selected_classes = []
        selected_probs = []
        while len(selected_boxes) < max_detections:
            # Select the prediction with the highest probability.
            i = np.argmax(max_probs)
            if max_probs[i] < self.prob_threshold:
                break

            # Save the selected prediction
            selected_boxes.append(boxes[i])
            selected_classes.append(max_classes[i])
            selected_probs.append(max_probs[i])

            box = boxes[i]
            other_indices = np.concatenate((np.arange(i), np.arange(i+1,len(boxes))))
            other_boxes = boxes[other_indices]

            # Get overlap between the 'box' and 'other_boxes'
            x1 = np.maximum(box[0], other_boxes[:,0])
            y1 = np.maximum(box[1], other_boxes[:,1])
            x2 = np.minimum(box[0]+box[2], other_boxes[:,0]+other_boxes[:,2])
            y2 = np.minimum(box[1]+box[3], other_boxes[:,1]+other_boxes[:,3])
            w = np.maximum(0, x2 - x1)
            h = np.maximum(0, y2 - y1)

            # Calculate Intersection Over Union (IOU)
            overlap_area = w * h
            iou = overlap_area / (areas[i] + areas[other_indices] - overlap_area)

            # Find the overlapping predictions
            overlapping_indices = other_indices[np.where(iou > self.IOU_THRESHOLD)[0]]
            overlapping_indices = np.append(overlapping_indices, i)

            # Set the probability of overlapping predictions to zero, and udpate max_probs and max_classes.
            class_probs[overlapping_indices,max_classes[i]] = 0
            max_probs[overlapping_indices] = np.amax(class_probs[overlapping_indices], axis=1)
            max_classes[overlapping_indices] = np.argmax(class_probs[overlapping_indices], axis=1)

        assert len(selected_boxes) == len(selected_classes) and len(selected_boxes) == len(selected_probs)
        return selected_boxes, selected_classes, selected_probs

    def _extract_bb(self, prediction_output, anchors):
        assert len(prediction_output.shape) == 3
        num_anchor = anchors.shape[0]
        height, width, channels = prediction_output.shape
        assert channels % num_anchor == 0

        num_class = int(channels / num_anchor) - 5
        assert num_class == len(self.labels)

        outputs = prediction_output.reshape((height, width, num_anchor, -1))
        
        # Extract bouding box information
        x = (self._logistic(outputs[...,0]) + np.arange(width)[np.newaxis, :, np.newaxis]) / width
        y = (self._logistic(outputs[...,1]) + np.arange(height)[:, np.newaxis, np.newaxis]) / height
        w = np.exp(outputs[...,2]) * anchors[:,0][np.newaxis, np.newaxis, :] / width
        h = np.exp(outputs[...,3]) * anchors[:,1][np.newaxis, np.newaxis, :] / height

        # (x,y) in the network outputs is the center of the bounding box. Convert them to top-left.
        x = x - w / 2
        y = y - h / 2
        boxes = np.stack((x,y,w,h), axis=-1).reshape(-1, 4)

        # Get confidence for the bounding boxes.
        objectness = self._logistic(outputs[...,4])

        # Get class probabilities for the bounding boxes.
        class_probs = outputs[...,5:]
        class_probs = np.exp(class_probs - np.amax(class_probs, axis=3)[..., np.newaxis])
        class_probs = class_probs / np.sum(class_probs, axis=3)[..., np.newaxis] * objectness[..., np.newaxis]
        class_probs = class_probs.reshape(-1, num_class)

        assert len(boxes) == len(class_probs)
        return (boxes, class_probs)
    
    def predict_image(self, image):
        inputs = self.preprocess(image)
        prediction_outputs = self.predict(inputs)
        return self.postprocess(prediction_outputs)

    def preprocess(self, image):
        image = image.convert("RGB") if image.mode != "RGB" else image
        image = image.resize((416, 416))
        return image

    def predict(self, preprocessed_inputs):
        """Evaluate the model and get the output

        Need to be implemented for each platforms. i.e. TensorFlow, CoreML, etc.
        """
        raise NotImplementedError

    def postprocess(self, prediction_outputs):
        """ Extract bounding boxes from the model outputs.

        Args:
            prediction_outputs: Output from the object detection model. (H x W x C)

        Returns:
            List of Prediction objects.
        """
        boxes, class_probs = self._extract_bb(prediction_outputs, self.ANCHORS)

        # Remove bounding boxes whose confidence is lower than the threshold.
        max_probs = np.amax(class_probs, axis=1)
        index, = np.where(max_probs > self.prob_threshold)
        index = index[(-max_probs[index]).argsort()]

        # Remove overlapping bounding boxes
        selected_boxes, selected_classes, selected_probs = self._non_maximum_suppression(boxes[index], class_probs[index], self.max_detections)

        return [{'probability': round(float(selected_probs[i]), 8),
                 'tagId': int(selected_classes[i]),
                 'tagName': self.labels[selected_classes[i]],
                 'boundingBox': {
                     'left': round(float(selected_boxes[i][0]), 8),
                     'top': round(float(selected_boxes[i][1]), 8),
                     'width': round(float(selected_boxes[i][2]), 8),
                     'height': round(float(selected_boxes[i][3]), 8)
                 }
             } for i in range(len(selected_boxes))]

In [ ]:
# Suggested steps before inference: 
# 1. for an image of width and height being (w, h) pixels, resize image to (w', h'), where w/h = w'/h' and w' x h' = 262144
# 2. resize network input size to (w', h')
# 3. pass the image to network and do inference
# (4. if inference speed is too slow for you, try to make w' x h' smaller)
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from PIL import Image
import os
MODEL_FILENAME = 'model.pb'
LABELS_FILENAME = 'labels.txt'

class TFObjectDetection(ObjectDetection):
    """Object Detection class for TensorFlow
    """
    def __init__(self, graph_def, labels):
        super(TFObjectDetection, self).__init__(labels)
        self.graph = tf.Graph()
        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')
            
    def predict(self, preprocessed_image):
        inputs = np.array(preprocessed_image, dtype=np.float)[:,:,(2,1,0)] # RGB -> BGR

        with tf.Session(graph=self.graph) as sess:
            output_tensor = sess.graph.get_tensor_by_name('model_outputs:0')
            outputs = sess.run(output_tensor, {'Placeholder:0': inputs[np.newaxis,...]})
            return outputs[0]


def main(image_filename):
    # Load a TensorFlow model
    graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(MODEL_FILENAME, 'rb') as f:
        graph_def.ParseFromString(f.read())

    # Load labels
    with open(LABELS_FILENAME, 'r') as f:
        labels = [l.strip() for l in f.readlines()]

    od_model = TFObjectDetection(graph_def, labels)

    image = Image.open(image_filename)
    predictions = od_model.predict_image(image)
    total=len(predictions)
    if(total>0):
        objects["text"]=str(total)+" Object(s) detected..."
    else:
        objects["text"]="No objects detected"
    for i in predictions:
        filename=os.path.splitext(image_filename)[0]
        extract_name_plate_bbox(image_filename,i["boundingBox"]["left"],i["boundingBox"]["top"],i["boundingBox"]["width"],i["boundingBox"]["height"],filename+"_plate.jpg",i["probability"])
def predict(filename):
    main(filename)

In [ ]:
from cv2 import *
#OCR Utility Function.
def perform_OCR(plate,name):
    cv2.imwrite(name,plate)
    text=azureOCR(name)
    result["text"]=text
    window.mainloop()
    
#extract nameplate from the image.
def extract_name_plate_bbox(path,left,top,width,height,name,probability):      
    img=cv2.imread(path)
    x = int(left * img.shape[1])
    y = int(top * img.shape[0])
    x2 = x + int(width * img.shape[1])
    y2 = y + int(height * img.shape[0])
    #crop image at name plate
    plate=img[y:y2,x:x2]
    #resize the image.
    plate=cv2.resize(plate,(0,0),fx=2,fy=2)
    cv2.imwrite(path,img)
    img = cv2.rectangle(img, (x,y), (x2,y2), (0,0,255), 2)
    background = cv2.cvtColor(plate, cv2.COLOR_BGR2RGB)
    photo = ImageTk.PhotoImage(image = PIL.Image.fromarray(background)) 
    canvas = Canvas(window, width = plate.shape[1], height = plate.shape[0])
    canvas.create_image(0, 0, image=photo, anchor=tk.NW)
    canvas.pack(side=LEFT,padx=30)
    perform_OCR(plate,name)

In [ ]:
import os
import requests
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO

if 'COGNITIVE_SERVICE_KEY' in os.environ:
    subscription_key = os.environ['COGNITIVE_SERVICE_KEY']
else:
    print("Set the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.")
    sys.exit()

if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']

ocr_url = endpoint + "vision/v2.0/ocr"
params = {'language': 'unk', 'detectOrientation': 'true'}
def azureOCR(filename):
    extract["text"]="extracting text"
    image_data = open(filename, "rb").read()
    headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}
    response = requests.post(ocr_url, headers=headers, params=params, data = image_data)
    analysis = response.json()
    line_infos = [region["lines"] for region in analysis["regions"]]
    word_infos = []
    for line in line_infos:
        for word_metadata in line:
            for word_info in word_metadata["words"]:
                word_infos.append(word_info)
    word_infos
    text=""
    for word in word_infos:
        bbox = [int(num) for num in word["boundingBox"].split(",")]
        text += word["text"]
    return text

In [ ]:
def selectfile():
    filename = askopenfilename(title = "Select Image to perform LPR",filetypes = [("All pictures","*.jpg *.png")])
     # show an "Open" dialog box and return the path to the selected file
    background = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
    cv2.resize(background,(0,0),fx=2,fy=2)
    photo = ImageTk.PhotoImage(image = PIL.Image.fromarray(background)) 
    canvas["width"]=background.shape[1]
    canvas["height"]=background.shape[0]
    btn_blur.destroy()
    canvas.create_image(0, 0, image=photo, anchor=tk.NW)
    predict(filename)